# Stochastic Galerkin Method

## Overview

Consider the following PDE
$$
\left\{
\begin{aligned}
u_t(x,\,t,\,\xi)&=\mathcal L(u), \,\, &D\times(0,\,T]\times\mathbb R^d,\\
\mathcal B(u)&=0,  &\partial D\times(0,\,T]\times\mathbb R^d,\\
u\mid_{t=0}&=u_0, &D\times\mathbb R^d,
\end{aligned}
\right.
$$
where $D\subset\mathbb R^l$ is physical domain of the process,
$\mathcal L$ is differential operator,
$\mathcal B$ is the boundary condition operator,
$u_0$ is the initial condition,
$\xi$ is a random variable.

Let's consider the gPC projection $u_N$ of the solution
$$
u_N(x,\,t,\,\xi)=\sum_{i=0}^N\hat u_i(x,\,t)p_i(\xi),\qquad
\hat u_i(x,\,t)=\frac1{\gamma_i}\mathbb E[u(x,\,t,\,\xi)p_i(\xi)]
$$

We assume that this projection approximates the solution to PDE quite well when $N$ is not very large. 

We immediately seek a solution in the form
$$
v_N(x,\,t,\,\xi)=\sum_{i=0}^N\hat v_i(x,\,t)p_i(\xi)
$$
as in the classical Galerkin approach for deterministic equations.

So, we get the following system for $i=0,\,1,\,\ldots,\,N$
$$
\left\{
\begin{aligned}
\mathbb E[\partial_tv_N(x,\,t,\,\xi)p_i(\xi)]&=\mathbb E[\mathcal L(u)p_i(\xi)], \,\, &D\times(0,\,T]\times\mathbb R^d,\\
\mathbb E[\mathcal B(v_N)p_i(\xi)]&=0,  &\partial D\times(0,\,T]\times\mathbb R^d,\\
\hat v_i\mid_{t=0}&=\frac1{\gamma_i}\mathbb E[u_0p_i(\xi)], &D\times\mathbb R^d,
\end{aligned}
\right.
$$

In the multivariate case, 
the size  of the system will be $\dim\mathcal P_N={N+d\choose N}$.

## ODE

We will solve a very simple ODE to illustrate the method
$$
\frac{du(t,\,\xi)}{dt}=-\alpha(\xi)u(t,\,\xi),\qquad
u(0,\,\xi)=\beta,
$$
where $\beta$ is deterministic.

We assume that $\alpha$ has a normal distribution $\alpha\sim\mathcal N(\mu,\,\sigma^2)$.

So, we will use one-dimensional Hermite polynomials $\{H_k(\xi)\}$ as a basis with $\xi\sim\mathcal N(0,1)$.  

The ODE parameter $\alpha$ is a linear function $\alpha=\mu+\sigma\xi.$ Hence, we can have a projection of $\alpha$ 
$$
\alpha_N(\xi)=\sum_{i=0}^Na_iH_i(\xi),
$$
with 
$$
a_0=\mu,\quad a_1=\sigma,\quad a_i=0,\,\,i>1.
$$

In this particular case, we have an exact relation
$$
\alpha(\xi)=\alpha_N(\xi),\quad N>1.
$$

In the similar fashion, we obtain a projection for $\beta$
$$
\beta_N(\xi)=\sum_{i=0}^Nb_iH_i(\xi),
$$
with 
$$
b_0=\beta,\quad b_i=0,\,\,i>1.
$$

(we will not use the fact of truncating the series until the end of the solution, to show the general principle)

We seek a solution in the form of gPC N-degree approximation
$$
v_N(t,\,\xi)=\sum_{i=0}^N\hat v_i(t)H_i(\xi),
$$
and after substitution the expansions in
$$
\mathbb E\left[\frac{dv_N}{dt}H_k\right]=
\mathbb E[-\alpha_Nv_NH_k],
$$
we got equations on unknown coefficients $\hat v_k$
$$
\frac{d\hat v_k}{dt}=-
\frac1{\gamma_k}\sum_{i=0}^N\sum_{j=0}^N
\alpha_i\hat v_ie_{ijk},
$$
where
$$
e_{ijk}=
\mathbb E[H_i(\xi)H_j(\xi)H_k(\xi)].
$$

The constants $e_{ijk}$ can be ''easily'' calculated
$$
\left\{
\begin{aligned}
e_{ijk}&=\frac{i!\,j!\,k!}{(s-i)!\,(s-j)!\,(s-k)!},
\;
&&s=\frac{i+j+k}2\in\mathcal N, \,s\geq i,j,k\\
e_{ijk}&=0, &&\text{otherwise}
\end{aligned}
\right.
$$
Recall that for Hermite polynomials $\gamma_k=k!$.

Thus, we have a deterministic system of ODE on $\vec v=\{\hat v_i\}$ which can  be written in a vector form
$$
\frac{\vec v_k(t)}{dt}=A \vec v,
$$
with initial condition
$$
\vec v(0)=\vec b,\quad \vec b=\{b_i\},
$$
and system $(N+1)\times(N+1)$ matrix $A=\{A_{kj}\}_0^N$ with elements
$$
A_{kj}=-\frac1{\gamma_k}\sum_{i=0}^Na_ie_{ijk}.
$$

In our simple case, 
$$
A_{kj}=-\frac1{\gamma_k}(\mu e_{0jk}+\sigma e_{1jk}),
$$
so, $A$ is tridiagonal,
$$
A_{j,j}=-\mu,
$$
$$
A_{j+1,j}=-\sigma,
$$
$$
A_{j,j+1}=-(j+1)\sigma.
$$

We can calculate the gPC projection coefficients of the solution for different $N$
$$
\vec v=e^{At}\vec b.
$$

Also, we can find the exact solution to this problem
$$
u=\beta e^{-\alpha t}=\beta e^{-\mu t}e^{-\sigma\xi t}.
$$

The gPC projection 
coefficients of this kind of expression we know from [Lecture 4](lecture-4.ipynb)
$$
\text{Pr}_Nu(t=1,\,\xi)=
\beta e^{-\mu+\sigma^2/2}\sum_{k=0}^N\frac{(-\sigma)^k}{k!}H_k(\xi).
$$

So, we can compare Stochastic Galerkin Method solution with the exact one choosing the different sets of parameters.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import scipy.sparse.linalg as SLA
import scipy.sparse as SP


def calc_A(N, mu=1, sigma=2):
    data = np.empty((3, N))
    data[0, ...] = mu
    data[1, ...] = sigma
    data[2, ...] = sigma*np.arange(N)
    return SP.dia_matrix((-data, [0, -1, 1]), shape=(N, N)).tocsc()


def calc_v_0(N, b0=1.0):
    return SP.dia_matrix(([b0], 0), shape=(N, 1)).tocsc()


def solution_coeffs(A, v_0, t):
    return SLA.expm(A*t).dot(v_0).toarray().flatten()


def solution(A, v_0, xi, T=1, num_t=100):
    t = np.linspace(0, T, num_t)
    sol = np.empty(num_t)
    N = A.shape[0]
    H_k = np.empty(N)
    for i in range(N):
        H_k[i] = p_herm(i, xi)

    for i, ti in enumerate(t):
        sol[i] = SLA.expm(A*ti).dot(v_0).toarray().flatten().dot(H_k)

    return sol


def exact_solution(N, mu, sigma):
    sol = np.empty(N)
    sol[0] = 1.0
    for i in range(1, N):
        sol[i] = sol[i-1]*(-sigma)/i
    return sol*np.exp(-mu + sigma*sigma/2.0)


def plot_both_solutions(N, mu, sigma):
    plt.figure(figsize=(10,7))
    plt.plot(exact_solution(N, mu, sigma), label='exact')

    A = calc_A(N, mu, sigma)
    v_0 = calc_v_0(N)
    cf = solution_coeffs(A, v_0, 1.0)
    plt.plot(cf, label='SGm')
    plt.legend(loc='upper right')


from ipywidgets import interact, widgets

In [2]:
interact(plot_both_solutions,  
         N=widgets.IntSlider(min=1,max=25,step=1,value=5,continuous_update=True,description='# of terms (N)'),
         mu=widgets.FloatSlider(min=-5,max=5,step=.25,value=0,continuous_update=True,description=r'$\mu$'),
         sigma=widgets.FloatSlider(min=0,max=5,step=.25,value=2,continuous_update=True,description=r'$\sigma$')
        );

interactive(children=(IntSlider(value=5, description='# of terms (N)', max=25, min=1), FloatSlider(value=0.0, description='$\\mu$', max=5.0, min=-5.0, step=0.25), FloatSlider(value=2.0, description='$\\sigma$', max=5.0, step=0.25), Output()), _dom_classes=('widget-interact',))

## Hyperbolic Equations

Consider a simple wave equation
$$
\frac{\partial u(x,t,\xi)}{\partial t}=c(\xi)
\frac{\partial u(x,t,\xi)}{\partial x},\qquad
x\in(-1,\,1),\;\;t>0,
$$
where $c(\xi)$ is a random transport velocity.
The initial condition is
$$
u(x,0,\xi)=u_0(x,\xi).
$$

The boundary conditions depend on the sign of $c$
(this sign determines in which direction the wave propagates)
$$
\begin{align}
u(1,t,\xi)&=u_R(t,\xi),& c(\xi)&>0,
\\
u(-1,t,\xi)&=u_L(t,\xi),& c(\xi)&<0.
\end{align}
$$
In this behavior, the stochastic equation differs significantly from the classical one.

Let us use normalized polynomials
$$
\mathbb E[p_i(\xi)p_j(\xi)]=\delta_{ij},
$$
and we haven't specify them yet. 

We seek the solution in the form of gPC expansion
$$
v_N(x,t,\xi)=\sum_{i=0}^N\hat v_N(x,t)p_i(\xi),
$$
and projecting the equation
$$
\mathbb E\left[
\frac{\partial u(x,t,\xi)}{\partial t}p_k(\xi)\right]=
\mathbb E\left[c(\xi)
\frac{\partial u(x,t,\xi)}{\partial x}p_k(\xi)\right],
$$
on the $k^\text{th}$ element of gPC basis, we obtain
$$
\frac{\partial\hat v_k(x,t)}{\partial t}=
\sum_{i=0}^Na_{ki}
\frac{\partial\hat v_i(x,t)}{\partial x},
$$
where
$$
a_{ki}=\mathbb E[c(\xi)p_k(\xi)p_i(\xi)].
$$

Again, we got a system of deterministic equations with the matrix $A=a_{ik}$, 
$$
\frac{\partial\vec v(x,t)}{\partial t}=
A
\frac{\partial\vec v(x,t)}{\partial x}.
$$
Note that now the system matrix is symmetric, $A^T=A$, so a complete set of real eigenvalues and eigenfunctions exists.
There exists an orthogonal matrix $S^T=S^{-1}$
such that $\Lambda=S^TAS$ is diagonal
$$
\Lambda=\mathop{\text{diag}}
(\underbrace{\lambda_0,\,\dots,\,\lambda_{j_+}}_{>0},
0,\,\ldots,0,\,
\underbrace{\lambda_{j_-},\ldots,\,\lambda_N}_{<0}).
$$

The sign of the eigenvalue is very important, since it determines which of the boundary conditions we will use (see the
[paper](https://utah.pure.elsevier.com/en/publications/galerkin-method-for-wave-equations-with-uncertain-coefficients) for details).

Denote $\vec q=\{q_0,\,q_1,\,\ldots,\,q_N\}^T=S^T\vec v $ and we obtain independent equations on $\vec q$ 
$$
\frac{\partial\vec q(x,t)}{\partial t}=
\Lambda
\frac{\partial\vec q(x,t)}{\partial x}.
$$
Boundary conditions depend on sign of eigenvalues
$$
\begin{align}
q_j(1,t)&=\sum_{k=0}^Ns_{kj}\hat u_k(1,t),&
j&=0,\,\dots,\,j_+,& (\lambda_j&>0),
\\
q_j(-1,t)&=\sum_{k=0}^Ns_{kj}\hat u_k(-1,t),&
j&=j_-,\ldots,\,N,& (\lambda_j&<0).
\end{align}
$$

### Error bound

The solution to the gPC Galerkin system converges to the exact solution. In fact, the following error bound was estimated (see the
[paper](https://utah.pure.elsevier.com/en/publications/galerkin-method-for-wave-equations-with-uncertain-coefficients))
$$
\mathbb E\left[\|u-v_N\|_{L_2}^2\right]\leq\frac C{N^{2m-1}}t,
$$
where $m>0$ depends on the smoothness of $u$ in terms of $\xi$.

## Diffusion Equations

Let us consider a time-dependent stochastic diffusion equation
$$
\left\{
\begin{align}
\frac{\partial u(x,t,\xi)}{\partial t}
=
\mathop{\text{div}}
\bigl(k(x,\xi)\mathop{\text{grad}}(u)\bigr)
+f(t,x),\qquad x\in D,\\
u(x,0,\xi)=u_0(x),\qquad u\mid_{\partial D}=0,
\end{align}
\right.
$$

We assume that $k$ has the form
$$
k(x,\xi)=\hat k_0(x)+\sum_{i=1}^d\hat k_i(x)\xi_i,
$$
where $\xi=(\xi_1,\,\xi_2,\,\ldots,\,\xi_d)$ are i.i.d. r.v.
Some ways of representing a random field as a finite combination of random variables will be considered in the following lectures.

For the problem to be well posed, we require
$$
k(x,\xi)\geq k_{\text{min}}.
$$

As usual, we seek the solution in the form of
$$
v_N(t,x,\xi)=\sum_{|k|=0}^N\hat v_k(t,x)p_k(\xi),
$$
where $k$ is multi-index and we use normalized polynomials
$$
\mathbb E[p_i(\xi)p_j(\xi)]=\delta_{ij}.
$$
For the simplicity, we will use single-index $i$ instead of $k$
$$
v_N(t,x,\xi)=\sum_{i=1}^M\hat v_i(t,x)p_i(\xi),
$$
where
$$
M={N+d\choose N}.
$$

Substituting the gPC expansion of the solution form to the equation and projecting the resulting equation, we get
$$
\frac{\partial \hat v_k(t,x)}{\partial t}=
\sum_{i=0}^d
\sum_{j=1}^M
\mathop{\text{div}}
\bigl(\hat k_i(x)\mathop{\text{grad}}(\hat v_j)\bigr)e_{ijk}
+\hat f_k(t, x)=
$$
$$
\sum_{j=1}^M
\mathop{\text{div}}
\bigl(a_{jk}(x)\mathop{\text{grad}}(\hat v_j)\bigr)
+\hat f_k(t, x),
$$
where
$$
e_{ijk}=
\mathbb E[\xi_ip_jp_k]=
\int z_ip_j(z)p_k(z)\,dF_\xi(z),
$$
and
$$
a_{jk}(x)=
\sum_{i=0}^d\hat k_i(x)e_{ijk}.
$$

The equation presented in the vector form 
$$
\frac{\partial\vec v(t,x)}{\partial t}=
\mathop{\text{div}}
\bigl(A(x)\mathop{\text{grad}}(\hat v_j)\bigr)
+\hat f,
$$
$$
\vec v(0,x)=\vec v_0(x),\quad \vec v\mid_{\partial D},
$$
where $A(x)=\{a_{ij}(x)\}$ is symmetric,
$\vec v=\{\hat v_k(t,x)\}$,
$\vec f=\{\hat f_k(t,x)\}$.

This deterministic system of equations is usually integrated numerically.

# Stochastic Collocation Method

In deterministic numerical analysis, collocation methods require that the residue of the governing equations to be zero at discrete nodes in the computational domain.
The nodes are called **collocation** points.
We can extend this idea to the stochastic simulations.

Consider a PDE
$$
\left\{
\begin{aligned}
u_t(x,\,t,\,\xi)&=\mathcal L(u), \,\, &D\times(0,\,T]\times I_\xi,\\
\mathcal B(u)&=0,  &\partial D\times(0,\,T]\times I_\xi,\\
u\mid_{t=0}&=u_0, &D\times I_\xi,
\end{aligned}
\right.
$$
where $I_\xi$ is the support of $\xi$.


Let $\Theta_M=\{\xi^{(j)}\}^M_{j=1}\subset I_\xi$
be a set of nodes in the random space.
Then in the collocation method, for all $j = 1,...,M$, we enforce the PDE at the node $\xi^{(j)}$ by solving
$$
\left\{
\begin{aligned}
u_t(x,\,t,\,\xi^{(j)})&=\mathcal L(u), \,\, &D\times(0,\,T],\\
\mathcal B(u)&=0,  &\partial D\times(0,\,T],\\
u\mid_{t=0}&=u_0, &D.
\end{aligned}
\right.
$$

For each $j$ it is a deterministic problem.
Let $u^{(j)} = u(\cdot,\xi^{(j)})$ be the  set of solutions of the above problem. 
We can extract useful information about $u(\xi)$ from this set.

**Definition (Stochastic collocation).** *Let  $\Theta_M = \{\xi^{(j)}\}^M_{j=1} ⊂ I_\xi$ be a set of (prescribed) nodes in the random space, where $M ≥ 1$ is the number of nodes,
and let $\{u^{(j)}\}^M_{j=1}$ be the solution of the governing equation.
Then find $w(\xi) ∈  \mathcal P(\xi)$ in a proper polynomial space  $\mathcal P(\xi)$
such that $w(\xi)$ is an approximation to the true solution $u(\xi)$ in the sense that $\|w(\xi) - u(\xi)\|$ is sufficiently small in a strong norm defined on $I_\xi$.*

Typically, the $L_p$ norm is used.

Convergence of stochastic collocation thus requires
$$
\|w(\xi) - u(\xi)\|\to0,\qquad M\to\infty.
$$

There exist two major approaches for high-order stochastic collocation: the *interpolation approach*
and the *discrete projection approach* (the *pseudospectral approach*).

## Interpolation approach

We have met this approach before -- find a polynomial $w$ form a given space $\mathcal P$ of polynomials such that $w(\xi^{(j)})=u^{(j)}$.

One can use *Lagrange interpolation approach* for this
$$
w(\xi)=\sum_{j=1}^Mu(\xi^{(j)})L_j(\xi)
$$
where
$$
L_j(\xi^{(i)})=\delta_{ij}.
$$
are the Lagrange interpolating polynomials.
In the multivariate case the theory is not so clear as in 1D. 
For example, 
existence of Lagrange interpolating polynomials for any given set of notes are not well understood.

The other way is a matrix inversion approach, where we prescribe the polynomial interpolating basis first. For example, let us use a set of gPC polynomial bases  $p_k(\xi)$ and construct
$$
w_N(\xi)=\sum_{|k|=0}^N\hat w_kp_k(\xi).
$$
The interpolation condition $w(\xi^{(j)})=u^{(j)}$
results in the following problem
for the unknown expansion coefficients
$$
A^T\hat w=u,
$$
where
$$
A=p_k(\xi^{(j)})
$$
is the Vandermonde-like coefficient matrix,
$\hat w$ is the vector of the expansion coefficients.

### Node selection

A typical strategies for node selection are

- Tensor product
- Random of pseudo-random (with low-discrepancy) point set
- Sparse grid


## Discrete Projection: Pseudospectral Approach

Recall, that in 1D the cubature rule, it is an integration rule that seeks to approximate an integral
$$
\int f(z)\,\text{d}F_Z(z)
$$
by
$$
U^Q[f]:=\sum_{j=1}^Qf(z^{(j)})\alpha^{(j)},
$$
where $z^{(j)}$, $\alpha^{(j)}$ are the nodes
and their corresponding weights.
The integration rule is convergent if it converges to the integral as $Q\to\infty$.

An integration rule of degree $m$ implies that the approximation $U^Q$ is exact for any integrand $f$ that is a polynomial of degree up to $m$ and is not exact for at least one polynomial of degree $m + 1$.

The coefficients $\hat u_k$ of the
orthogonal gPC projection of $u$
$$
u_N(\xi)={\text{Pr}}_Nu=
\sum_{|k|=0}^N\hat u_kp_k(k)
$$
are expressed in terms of the corresponding integrals
$$
\hat u_k=
\frac1{\gamma_k}
\int u(z)p_k(z)\,\text{d}F_\xi(z).
$$

The idea is to approximate
the integrals in the expansion coefficients 
by an integration rule.

The discrete projection of the solution of the PDE, which we considered above, is
$$
w_N(\xi)=
\sum_{|k|=0}^N\hat w_kp_k(k)
$$
where the expansion coefficients are
$$
\hat w_k=\frac1{\gamma_k}
U^Q[u(\xi)p_k(\xi)]=
\frac1{\gamma_k}\sum_{j=1}^Qu(z^{(j)})p_k(z^{(j)})\alpha^{(j)}.
$$

**Proposition**
*For $u(\xi)\in L^2_{dF_\xi}(I_\xi)$,
let $u_N(\xi)$ be the gPC projection of $u$ and let
$w_N(\xi)$ be the discrete gPC projection for $u$.
Assume that the cubature rule $U^Q$ is convergent;
then as $Q\to\infty$, $\hat w_k\to\hat u_k$, for all $|k|\leq N$, and
$$
w_N(\xi)\to u_N(\xi),\quad\forall \xi.
$$*

We can estimate the error induced by $w_N$.

**Proposition**
*For $u(\xi)\in L^2_{dF_\xi}(I_\xi)$,
let $u_N(\xi)$ be the gPC projection of $u$ and let
$w_N(\xi)$ be the discrete gPC projection for $u$.
Then,
$$
\|w_N(\xi)-u(\xi)\|_{L^2_{dF_\xi}}\leq
\|u_N(\xi)-u(\xi)\|_{L^2_{dF_\xi}}+
\|w_N(\xi)-u_N(\xi)\|_{L^2_{dF_\xi}}.
$$*

The first term of the error is the gPC projection error induced by using finite-order ($N^{\text{th}}$-order) polynomials. The second term of the error is the difference between the continuous gPC projection and the discrete projection and is caused by using a cubature rule with finite accuracy.

We can express the second error as
$$
\epsilon^Q_N:=
\|w_N(\xi)-u_N(\xi)\|_{L^2_{dF_\xi}}
$$
$$=
\left(\sum_{|k|=0}^N(\hat w_k-\hat u_k)^2\gamma_k\right)^{\frac12}.
$$

 A distinct feature of the discrete projection approach is that one can compute only the coefficients that are important for a given problem without evaluating the rest of the expansion coefficients. This may happen, for example, when global sensitivity is required for some input random variables $\xi$. This is in contrast to the gPC Galerkin method, where all the gPC coefficients are coupled and solved simultaneously.